In [5]:
!pip install transformers torch bitsandbytes

In [27]:
!pip install -i https://pypi.org/simple/ bitsandbytes
!pip install openai, requests, gnews
!pip install accelerate pandas

Looking in indexes: https://pypi.org/simple/
ERROR: Invalid requirement: 'openai,'


In [26]:
# Necessary imports
import sys
import os
import openai, requests, gnews
import pandas as pd

ModuleNotFoundError: No module named 'openai'

In [10]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from transformers import BitsAndBytesConfig

In [12]:
from huggingface_hub import notebook_login
notebook_login(access_token)

In [13]:
# Setup environment variables for cache and token
os.environ['HF_HOME'] = '/content/cache'
os.environ['TRANSFORMERS_CACHE'] = '/content/cache'
os.environ['HF_DATASETS_CACHE'] = '/content/cache/datasets'
os.environ['HF_METRICS_CACHE'] = '/content/cache/metrics'
os.environ['HF_MODULES_CACHE'] = '/content/cache/modules'
os.environ['HF_TOKEN'] = access_token

In [14]:
# Create efficient model
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [4]:
model_id = "meta-llama/Llama-2-7b-hf"
tokenizer = AutoTokenizer.from_pretrained(model_id, token=access_token)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    token=access_token,
    quantization_config=bnb_config
    )


NameError: name 'AutoTokenizer' is not defined

In [5]:
text_generator = pipeline(
    "text-generation",
    model=model_id,
    tokenizer=tokenizer,
    max_new_tokens=8
    )

NameError: name 'pipeline' is not defined

In [ ]:
# Model is loaded above
# Test prompt
def get_response(prompt):
    response = text_generator(prompt)
    gen_text = response[0]['generated_text']
    return gen_text

prompt = "List a brief history of Napolean"
llama_response = get_response(prompt)
print(llama_response)

In [18]:
# Mount your google drive in google colab
from google.colab import drive
drive.mount('/content/drive')

# Insert the directoryimport sys
sys.path.insert(0,'/content/drive/MyDrive/Colab Notebooks')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [32]:
# Import files
from script_with_llama import get_news_articles, analyze_relationships

ModuleNotFoundError: No module named 'gnews'

In [30]:
# Test it out using said modules

query = input("Enter the search query for news articles: ")
articles = get_news_articles(query)

# Add try-catch if no articles found (error processing)
if not articles:
    print("No articles found for the query.")
    return

# Randomly pick 5 articles from the fetched articles
sampled_articles = random.sample(articles, min(5, len(articles)))

    # Concatenate all article contents
combined_text = ""
for article in sampled_articles:
    content = article['description'] or article['content']
    if content:
        combined_text += content + "\n\n"

# Analyze the combined text
results = analyze_relationships(combined_text)

# Parse the results and create a DataFrame
parsed_results = []
for result in results:
    lines = result.split("\n")
    for line in lines:
        if line:
            parts = line.split(',')
            if len(parts) == 3:
                parsed_results.append(parts)

Enter the search query for news articles: hello


NameError: name 'get_news_articles' is not defined

In [ ]:
df = pd.DataFrame(parsed_results, columns=['Company1', 'Company2', 'RelationshipType'])
df.to_csv('company_relationships.csv', index=False)
print("Results saved to 'company_relationships.csv'.")

**Suggestions for improvements.**

Work to see how we can define edge-weights for the current model. More possible ideas for fine-tuning the LLM prompt to get cleaner results.

Now that we have a baseline of company-company relationships, how can we define edge weights and upstream/downstream features to create a baseline graph.

**Changes made**

Setting this environment up in Google Colab so that it can be used by anyone. Also added the functionality above for testing so that you can try to debug this script by yourself.